# Classification Model - XGBoost

In [687]:
#Importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
from itertools import combinations
from scipy.stats import chi2_contingency
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, make_scorer
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV

In [688]:
print(os.getcwd())

c:\Users\ntama\Documentos\Data Science\Codecademy\projects\Date-A-Scientist - Final Project\OKCupid-Date-A-Scientist-Starter\date-a-scientist-project\notebooks


## Importing predictors and target - train, test and validation subsets

In [689]:
X_train = pd.read_csv("../data/X_train.csv", index_col=False)
X_train = X_train.loc[:, ~X_train.columns.str.contains('^Unnamed')]

X_val = pd.read_csv("../data/X_val.csv", index_col=False)
X_val = X_val.loc[:, ~X_val.columns.str.contains('^Unnamed')]

X_test = pd.read_csv("../data/X_test.csv", index_col=False)
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]

y_train = pd.read_csv("../data/y_train.csv", index_col=False)
y_train = y_train.loc[:, ~y_train.columns.str.contains('^Unnamed')]

y_val = pd.read_csv("../data/y_val.csv", index_col=False)
y_val = y_val.loc[:, ~y_val.columns.str.contains('^Unnamed')]

y_test = pd.read_csv("../data/y_test.csv", index_col=False)
y_test = y_test.loc[:, ~y_test.columns.str.contains('^Unnamed')]

These are the features with the most importance. They are being left as markdown in case it is necessary to drop them.

X_train = X_train.drop(['sex'], axis=1)
X_val = X_val.drop(['sex'], axis=1)
X_test = X_test.drop(['sex'], axis=1)

X_train = X_train.drop(['height'], axis=1)
X_val = X_val.drop(['height'], axis=1)
X_test = X_test.drop(['height'], axis=1)

X_train = X_train.drop(['age'], axis=1)
X_val = X_val.drop(['age'], axis=1)
X_test = X_test.drop(['age'], axis=1)

Checking data types for features in all three predictor sets

In [690]:
print(X_train.dtypes[X_train.dtypes == 'object']) 
print(X_val.dtypes[X_val.dtypes == 'object'])    
print(X_test.dtypes[X_test.dtypes == 'object'])   


Series([], dtype: object)
Series([], dtype: object)
Series([], dtype: object)


## Checking class imbalance

In [691]:
print(y_train.value_counts())

body_type
0            35519
1             6443
Name: count, dtype: int64


### RandomOverSampler on train set for class balance

In [692]:
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

## Model initialization

Calculating weights for each class 

In [693]:
# Making sure y_train and y_val is a series for calculating scale_pos_weight
if isinstance(y_train_res, pd.DataFrame):
    y_train_res = y_train_res.squeeze()

if isinstance(y_val, pd.DataFrame):
    y_val = y_val.squeeze()


# Ratios for imbalanced classes
pos = np.sum(y_train == 1)
neg = np.sum(y_train == 0)
scale_pos_weight = neg / pos


c:\Users\ntama\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


### Base model definition with basic parameters

In [694]:
model1 = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,
    n_jobs=-1,
    random_state=42,
    eval_metric='logloss'
)

### Training the model with val set

In [695]:
model1.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_val, y_val)],
    verbose=True
)

[0]	validation_0-logloss:0.66139
[1]	validation_0-logloss:0.64525
[2]	validation_0-logloss:0.63674
[3]	validation_0-logloss:0.63229
[4]	validation_0-logloss:0.62925
[5]	validation_0-logloss:0.62758
[6]	validation_0-logloss:0.62666
[7]	validation_0-logloss:0.62597
[8]	validation_0-logloss:0.62517
[9]	validation_0-logloss:0.62483
[10]	validation_0-logloss:0.62404
[11]	validation_0-logloss:0.62371
[12]	validation_0-logloss:0.62359
[13]	validation_0-logloss:0.62321
[14]	validation_0-logloss:0.62297
[15]	validation_0-logloss:0.62256
[16]	validation_0-logloss:0.62252
[17]	validation_0-logloss:0.62201
[18]	validation_0-logloss:0.62194
[19]	validation_0-logloss:0.62179
[20]	validation_0-logloss:0.62177
[21]	validation_0-logloss:0.62177
[22]	validation_0-logloss:0.62157
[23]	validation_0-logloss:0.62156
[24]	validation_0-logloss:0.62158
[25]	validation_0-logloss:0.62149


c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:30:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[26]	validation_0-logloss:0.62146
[27]	validation_0-logloss:0.62131
[28]	validation_0-logloss:0.62136
[29]	validation_0-logloss:0.62128
[30]	validation_0-logloss:0.62091
[31]	validation_0-logloss:0.62091
[32]	validation_0-logloss:0.62091
[33]	validation_0-logloss:0.62053
[34]	validation_0-logloss:0.62052
[35]	validation_0-logloss:0.62045
[36]	validation_0-logloss:0.62027
[37]	validation_0-logloss:0.62022
[38]	validation_0-logloss:0.62001
[39]	validation_0-logloss:0.62009
[40]	validation_0-logloss:0.62012
[41]	validation_0-logloss:0.62012
[42]	validation_0-logloss:0.61983
[43]	validation_0-logloss:0.61956
[44]	validation_0-logloss:0.61960
[45]	validation_0-logloss:0.61934
[46]	validation_0-logloss:0.61926
[47]	validation_0-logloss:0.61937
[48]	validation_0-logloss:0.61929
[49]	validation_0-logloss:0.61945
[50]	validation_0-logloss:0.61948
[51]	validation_0-logloss:0.61948
[52]	validation_0-logloss:0.61889
[53]	validation_0-logloss:0.61864
[54]	validation_0-logloss:0.61843
[55]	validatio

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, ...)

### Predictions and metrics report

In [696]:
y_val_pred = model1.predict(X_val)

# Metric report
print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.91      0.65      0.76      7611
           1       0.25      0.65      0.36      1381

    accuracy                           0.65      8992
   macro avg       0.58      0.65      0.56      8992
weighted avg       0.81      0.65      0.70      8992

[[4930 2681]
 [ 478  903]]


### Showing feature importance

In [701]:
print(X_train.columns)

Index(['age', 'height', 'age_group', 'height_group', 'diet_anything',
       'diet_halal', 'diet_kosher', 'diet_other', 'diet_vegan',
       'diet_vegetarian', 'drinks_a little', 'drinks_a lot',
       'drinks_not at all', 'drugs_never', 'drugs_often', 'drugs_sometimes',
       'smokes_drinks_no_a little', 'smokes_drinks_no_a lot',
       'smokes_drinks_no_not at all', 'smokes_drinks_yes_a little',
       'smokes_drinks_yes_a lot', 'smokes_drinks_yes_not at all',
       'drinks_drugs_a little_never', 'drinks_drugs_a little_often',
       'drinks_drugs_a little_sometimes', 'drinks_drugs_a lot_never',
       'drinks_drugs_a lot_often', 'drinks_drugs_a lot_sometimes',
       'drinks_drugs_not at all_never', 'drinks_drugs_not at all_often',
       'drinks_drugs_not at all_sometimes', 'smokes_drugs_no_never',
       'smokes_drugs_no_often', 'smokes_drugs_no_sometimes',
       'smokes_drugs_yes_never', 'smokes_drugs_yes_often',
       'smokes_drugs_yes_sometimes', 'sex', 'smokes', 'risky_beh

In [697]:
model1.feature_importances_

array([0.01398232, 0.01236286, 0.        , 0.        , 0.01341829,
       0.00798391, 0.0099363 , 0.01688178, 0.01451469, 0.02168611,
       0.0139861 , 0.01250083, 0.01357421, 0.02069929, 0.01640162,
       0.01532008, 0.06201674, 0.01118893, 0.02020106, 0.01518577,
       0.01249114, 0.013677  , 0.01354309, 0.00536935, 0.01682473,
       0.0147698 , 0.00397785, 0.01656724, 0.01168715, 0.00446333,
       0.01024144, 0.02025638, 0.01626467, 0.01245941, 0.01558495,
       0.00926712, 0.01219338, 0.45117152, 0.01274144, 0.01460818],
      dtype=float32)

### Hyperparameter tuning with GridSearchCV
Focusing on high recall

In [698]:
model2 = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,
    n_jobs=-1,
    random_state=42,
    eval_metric='logloss'
)

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.2]
}

grid_search = GridSearchCV(
    estimator=model2,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
)


grid_search.fit(X_train_res, y_train_res)

c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:30:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:30:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:30:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ntama\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:30:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=-1, num_parallel_tree=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.2],
                         'learning_rate': [0.05, 0.1], 'max_depth': [3, 5],
                         'min_child_weight': [1, 3], 'n_estimators': [100, 200],
                         'subsample': [0.8, 1.0]},
             scoring='recall')

In [699]:
best_model = grid_search.best_estimator_

y_val_pred = best_model.predict(X_val)

print("Best hyperparameters: ", grid_search.best_params_)
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))
print("\nConfusion Matrix")
print(confusion_matrix(y_val, y_val_pred))

Best hyperparameters:  {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.8}

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.64      0.75      7611
           1       0.25      0.68      0.37      1381

    accuracy                           0.65      8992
   macro avg       0.58      0.66      0.56      8992
weighted avg       0.81      0.65      0.69      8992


Confusion Matrix
[[4866 2745]
 [ 447  934]]


### Finding the optimal prediction threshold to maximize Recall and F1

In [700]:
from sklearn.metrics import precision_recall_fscore_support

y_proba = best_model.predict_proba(X_val)[:, 1]  # Probabilidad de clase positiva

thresholds = np.arange(0.1, 0.9, 0.05)
for thresh in thresholds:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred_thresh, average='binary')
    print(f"Threshold: {thresh:.2f} | Accuracy: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f}")


Threshold: 0.10 | Accuracy: 0.154 | Recall: 0.997 | F1: 0.266
Threshold: 0.15 | Accuracy: 0.155 | Recall: 0.993 | F1: 0.268
Threshold: 0.20 | Accuracy: 0.156 | Recall: 0.986 | F1: 0.269
Threshold: 0.25 | Accuracy: 0.167 | Recall: 0.953 | F1: 0.285
Threshold: 0.30 | Accuracy: 0.192 | Recall: 0.881 | F1: 0.316
Threshold: 0.35 | Accuracy: 0.214 | Recall: 0.797 | F1: 0.338
Threshold: 0.40 | Accuracy: 0.238 | Recall: 0.743 | F1: 0.361
Threshold: 0.45 | Accuracy: 0.249 | Recall: 0.708 | F1: 0.368
Threshold: 0.50 | Accuracy: 0.254 | Recall: 0.676 | F1: 0.369
Threshold: 0.55 | Accuracy: 0.261 | Recall: 0.637 | F1: 0.370
Threshold: 0.60 | Accuracy: 0.268 | Recall: 0.541 | F1: 0.359
Threshold: 0.65 | Accuracy: 0.293 | Recall: 0.322 | F1: 0.307
Threshold: 0.70 | Accuracy: 0.332 | Recall: 0.158 | F1: 0.214
Threshold: 0.75 | Accuracy: 0.363 | Recall: 0.062 | F1: 0.105
Threshold: 0.80 | Accuracy: 0.371 | Recall: 0.017 | F1: 0.032
Threshold: 0.85 | Accuracy: 0.400 | Recall: 0.003 | F1: 0.006
